Chapter 19
Looping and Aggregation

In [ ]:
# deletes variables
%reset -f

19.1 For Loops



In [ ]:
import pandas as pd

In [ ]:
def tweak_siena_pres(df):
    def int64_to_uint8(df_):
        cols = df_.select_dtypes('int64')
        return (df_
                .astype({col:'uint8' for col in cols}))


    return (df
     .rename(columns={'Seq.':'Seq'})    # 1
     .rename(columns={k:v.replace(' ', '_') for k,v in
        {'Bg': 'Background',
         'PL': 'Party leadership', 'CAb': 'Communication ability',
         'RC': 'Relations with Congress', 'CAp': 'Court appointments',
         'HE': 'Handling of economy', 'L': 'Luck',
         'AC': 'Ability to compromise', 'WR': 'Willing to take risks',
         'EAp': 'Executive appointments', 'OA': 'Overall ability',
         'Im': 'Imagination', 'DA': 'Domestic accomplishments',
         'Int': 'Integrity', 'EAb': 'Executive ability',
         'FPA': 'Foreign policy accomplishments',
         'LA': 'Leadership ability',
         'IQ': 'Intelligence', 'AM': 'Avoid crucial mistakes',
         'EV': "Experts' view", 'O': 'Overall'}.items()})
     .astype({'Party':'category'})  # 2
     .pipe(int64_to_uint8)  # 3
     .assign(Average_rank=lambda df_:(df_.select_dtypes('uint8') # 4
                 .sum(axis=1).rank(method='dense').astype('uint8')),
             Quartile=lambda df_:pd.qcut(df_.Average_rank, 4,
                 labels='1st 2nd 3rd 4th'.split())
            )
    )

In [ ]:


pd.set_option('display.max_rows', 10)             
url = 'https://github.com/mattharrison/datasets/raw/master/data/'\
    'siena2018-pres.csv'
df = pd.read_csv(url, index_col=0)

pres = tweak_siena_pres(df)


In [ ]:
pres

In [ ]:
pres.info()

In [ ]:
# iteration over columns (col_name, series) tuple
for col_name, col in pres.iteritems():
    print(col_name, type(col))
    #break
    

In [ ]:
for idx, row in pres.iterrows():
    print(idx, type(row))
    break

In [ ]:
# iteration over rows as namedtuple (index as first item)

for tup in pres.itertuples():
    print(tup[0], len(tup), tup.Party)
    break


19.2 Aggregations

In [ ]:
scores = (pres
          .loc[:, 'Background':'Average_rank'])

scores.sum(axis='columns')/len(scores.columns)





In [ ]:
scores.mean(axis=1)

In [ ]:
# Why ?

pres.agg(['count', 'size', 'sum', lambda col:col.loc[1]])

In [ ]:
pres.agg({'Luck':['count', 'size'], 'Overall':['count', 'max']})

In [ ]:
pres.agg(Intelligence_count=('Intelligence','count'), Intelligence_size=('Intelligence', 'size') )


In [ ]:
pres.describe()

In [ ]:
pres.describe(include='all')

In [ ]:
(pres
 .select_dtypes('number')
 .pipe(lambda df_: df_.max(axis='columns')
       -df_.min(axis='columns'))
)

In [ ]:
(pres
 .select_dtypes('number')
 .apply(lambda row: row.max()-row.min(), axis='columns')
 )